In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!git clone https://github.com/logacc99/LPR.git

Cloning into 'LPR'...
remote: Enumerating objects: 283, done.
remote: Counting objects: 100% (283/283), done.
remote: Compressing objects: 100% (229/229), done.
remote: Total 283 (delta 54), reused 261 (delta 42), pack-reused 0
Receiving objects: 100% (283/283), 1.10 MiB | 7.55 MiB/s, done.
Resolving deltas: 100% (54/54), done.


In [ ]:
import os
os.symlink('/content/drive/MyDrive/lpr-assets/models','/content/LPR/models')

In [115]:
%cd /content/LPR

/content/LPR


In [5]:
!pip install -qr requirements.txt # install

     |████████████████████████████████| 596 kB 5.5 MB/s 


In [77]:
%load_ext autoreload
%autoreload 2
from full_flow import detect_batch, LP_Detector

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [79]:
lp_detector = LP_Detector()

Loading WPOD-NET...


Fusing layers... 


Loading Vehicle Detector...


Model Summary: 213 layers, 7225885 parameters, 0 gradients


Loading OCR Model...
Loading Type Predictor...
Initialize done.


In [109]:
from glob import glob
import cv2
import numpy as np
img_paths = glob('/content/drive/MyDrive/UFPR-ALPR-dataset/testing/**/*.png', recursive=True)
# img_paths = glob('/content/*.png', recursive=True)
img_paths = [path for path in img_paths if any([key in path for key in ['[01]','[10]','[20]','[30]']])]
# img_paths = [path for path in img_paths if '96' in path]
print(len(img_paths))
print(img_paths[0])

240
/content/drive/MyDrive/UFPR-ALPR-dataset/testing/track0133/track0133[01].png


In [ ]:
np.random.shuffle(img_paths)

In [111]:
save_dir = '/content/drive/MyDrive/yolov5/output/testing-9-1'
detect_batch(img_paths, lp_detector, save_dir = '/content/output')

True


 11%|█         | 26/240 [00:21<02:55,  1.22it/s]/content/LPR/full_flow.py:189: MatplotlibDeprecationWarning: Passing the block parameter of show() positionally is deprecated since Matplotlib 3.1; the parameter will become keyword-only in 3.3.
  else:
100%|██████████| 240/240 [03:33<00:00,  1.12it/s]


# Test accuracy

In [ ]:
%rm -rf /content/testing

In [ ]:
!unzip /content/testing.zip -d /content/testing

In [98]:
from glob import glob
import cv2
import numpy as np
import editdistance
gt_paths = glob('/content/drive/MyDrive/UFPR-ALPR-dataset/labels/ocr/testing/**/*.txt', recursive=True)
gt_paths = [path for path in gt_paths if any([key in path for key in ['[01]', '[05]', '[10]','[15]' ,'[20]', '[25]', '[30]']])]
pred_paths = glob('/content/output/**/*.txt', recursive=True)
print(len(gt_paths), len(pred_paths))

True
420 240


In [112]:
total = len(pred_paths)
from tqdm import tqdm
wrong_detection = []
wrong_count = 0
avg_acc = 0
for pred_path in tqdm(pred_paths):
    fname = pred_path.split('/')[-1]
    gt_path = '/content/drive/MyDrive/UFPR-ALPR-dataset/labels/ocr/testing/' + fname
    with open(gt_path, 'r') as f:
        anno = f.readline()
        gt_ocr = anno.split(' ')[-1]
        # print('GT:', gt_ocr)
    
    with open(pred_path, 'r') as f:
        pred_ocr = f.read()
        # print('Pred:', pred_ocr)

    gt_ocr = gt_ocr.replace('-','')
    if pred_ocr != 'None':
        pred_ocr = pred_ocr[:-3]

    if pred_ocr != gt_ocr:
        wrong_count +=1
        wrong_detection.append([fname, gt_ocr, pred_ocr])
    
    # Another metrics: Edit Distance
    ed = editdistance.eval(gt_ocr, pred_ocr)
    acc = 1-ed/len(gt_ocr)
    if acc < 0: acc = 0
    avg_acc += acc / len(gt_paths)

100%|██████████| 240/240 [00:00<00:00, 567.87it/s]


In [113]:
print('Wrong detections:', wrong_count)
print('Accuracy:', (total-wrong_count)/total)
print(avg_acc)

Wrong detections: 62
Accuracy: 0.7416666666666667
0.4867346938775503


In [114]:
wrong_detection = sorted(wrong_detection, key = lambda x: x[0])
for det in wrong_detection:
    print(det)

['track0091[01].txt', 'MLS5511', 'None']
['track0094[30].txt', 'AUY8870', 'None']
['track0095[01].txt', 'AVI2214', '4VI224']
['track0097[01].txt', 'AYP7941', 'A6P7941']
['track0097[10].txt', 'AYP7941', '4P794']
['track0097[20].txt', 'AYP7941', 'A7P7941']
['track0097[30].txt', 'AYP7941', 'AP794I']
['track0099[01].txt', 'APE4685', 'IPE4685']
['track0099[10].txt', 'APE4685', 'IPE54685']
['track0099[30].txt', 'APE4685', 'IIPE4685']
['track0102[01].txt', 'AHB1989', 'AH8I99']
['track0102[30].txt', 'AHB1989', 'AHB0989']
['track0104[10].txt', 'APL0100', 'PL0I00']
['track0106[20].txt', 'ABN8528', 'None']
['track0106[30].txt', 'ABN8528', 'None']
['track0107[01].txt', 'MJO0862', 'None']
['track0107[10].txt', 'MJO0862', 'RAND0N']
['track0107[20].txt', 'MJO0862', 'None']
['track0107[30].txt', 'MJO0862', 'RAIWID0W']
['track0108[01].txt', 'ARY9308', 'None']
['track0108[10].txt', 'ARY9308', 'None']
['track0108[20].txt', 'ARY9308', 'None']
['track0109[01].txt', 'AWZ7648', '4S']
['track0109[10].txt', 'A

In [ ]:
['track0091[01].txt', 'MLS5511', 'None']
['track0094[01].txt', 'AUY8870', 'A6']
['track0094[30].txt', 'AUY8870', 'None']
['track0095[01].txt', 'AVI2214', '4V224']
['track0097[01].txt', 'AYP7941', '46P794']
['track0097[10].txt', 'AYP7941', '4TP7941']
['track0097[20].txt', 'AYP7941', 'A7P7941']
['track0097[30].txt', 'AYP7941', '4P794I']
['track0099[01].txt', 'APE4685', 'IPE4685']
['track0099[20].txt', 'APE4685', 'AIPE4685']
['track0099[30].txt', 'APE4685', 'IPE4685']
['track0100[20].txt', 'ISK7086', 'I5K706']
['track0101[20].txt', 'AMR2189', 'AMR2109']
['track0102[01].txt', 'AHB1989', 'AHB1909']
['track0107[01].txt', 'MJO0862', 'None']
['track0107[10].txt', 'MJO0862', 'RAN1D0N']
['track0107[20].txt', 'MJO0862', 'None']
['track0107[30].txt', 'MJO0862', 'RAWD01W']
['track0108[01].txt', 'ARY9308', 'None']
['track0108[10].txt', 'ARY9308', 'None']
['track0108[20].txt', 'ARY9308', 'None']
['track0109[10].txt', 'AWZ7648', '']
['track0109[20].txt', 'AWZ7648', '5']
['track0109[30].txt', 'AWZ7648', '9']
['track0110[30].txt', 'BBO8514', 'B8O8514']
['track0111[20].txt', 'AIQ1056', 'AIQP1056']
['track0112[30].txt', 'APD4389', 'APO4389']
['track0114[01].txt', 'ASF5742', 'SF5742']
['track0114[20].txt', 'ASF5742', 'SF5742']
['track0115[30].txt', 'CBB1894', 'CBB1B94']
['track0116[10].txt', 'PYP1906', '']
['track0117[20].txt', 'AYE1896', 'None']
['track0118[20].txt', 'AWL4456', 'AWL4458']
['track0119[20].txt', 'IOZ3616', 'None']
['track0119[30].txt', 'IOZ3616', 'None']
['track0122[01].txt', 'MCA9954', 'CA9954']
['track0122[10].txt', 'MCA9954', 'CA9954']
['track0123[10].txt', 'ASG9143', 'A5G9I4']
['track0123[20].txt', 'ASG9143', 'I45IG9I43']
['track0123[30].txt', 'ASG9143', 'ASI9143']
['track0124[01].txt', 'MAW3186', 'HAW3186']
['track0124[10].txt', 'MAW3186', 'HAWF31P6']
['track0124[20].txt', 'MAW3186', 'MAW31P6']
['track0127[01].txt', 'AQW1379', 'T79']
['track0127[10].txt', 'AQW1379', 'A0WI79']
['track0127[20].txt', 'AQW1379', 'I379']
['track0127[30].txt', 'AQW1379', 'I1']
['track0128[01].txt', 'AWP0816', 'AMP0816']
['track0128[10].txt', 'AWP0816', 'AHP0816']
['track0128[20].txt', 'AWP0816', 'AMP0B16']
['track0128[30].txt', 'AWP0816', 'AHP0816']
['track0129[01].txt', 'MAK0198', 'PRP0NTAG90SSI']
['track0129[30].txt', 'MAK0198', 'NAK0198']
['track0130[20].txt', 'ALJ9348', '2ALJ9348']
['track0134[20].txt', 'ARG4938', 'None']
['track0134[30].txt', 'ARG4938', 'None']
['track0135[20].txt', 'BAS8216', 'BAS826']
['track0136[01].txt', 'AMO0663', 'AMU0663']
['track0136[20].txt', 'AMO0663', 'AM00663I']
['track0138[01].txt', 'AUG0936', '09036']
['track0140[01].txt', 'AVA1466', 'VAI466']
['track0140[30].txt', 'AVA1466', 'None']
['track0144[30].txt', 'AYL2104', 'AYL204']
['track0149[01].txt', 'AXW8502', 'None']
['track0149[10].txt', 'AXW8502', 'None']